In [8]:
from collections import defaultdict

labeler_names = [
    'race:black:labeler=james:validation',
    'race:black:labeler=dan:validation',
    'race:black:labeler=kayvon:validation',
]
labelers = Labeler.objects.filter(name__in=labeler_names)
tag = Tag.objects.get(name='black')
face_tags = FaceTag.objects.filter(
    tag=tag, labeler__id__in=labelers
).values('face__id', 'score', 'labeler__name')

labelers_pos = defaultdict(set)
labelers_neg = defaultdict(set)
for f in face_tags:
    labeler_name = f['labeler__name']
    if f['score'] > 0.5:
        labelers_pos[labeler_name].add(f['face__id'])
    else:
        labelers_neg[labeler_name].add(f['face__id'])
        
for l1 in labeler_names:
    for l2 in labeler_names:
        if l1 <= l2:
            continue
        print('{} and {}'.format(l1, l2))
        n_black_agree = len(labelers_pos[l1] & labelers_pos[l2])
        n_black_disagree = len(labelers_pos[l1] ^ labelers_pos[l2])
        print('  black: {} agreements, {} disagreements, {:0.1f}:1 ratio, {:0.1f}% disagreement'.format(
            n_black_agree, n_black_disagree, 
            n_black_agree / n_black_disagree,
            n_black_disagree / (n_black_agree + n_black_disagree) * 100
        ))
        n_not_black_agree = len(labelers_neg[l1] & labelers_neg[l2])
        n_not_black_disagree = len(labelers_neg[l1] ^ labelers_neg[l2])
        print('  not-black: {} agreements, {} disagreements, {:0.1f}:1 ratio, {:0.1f}% disagreement'.format(
            n_not_black_agree, n_not_black_disagree,
            n_not_black_agree / n_not_black_disagree,
            n_not_black_disagree / (n_not_black_agree + n_not_black_disagree) * 100
        ))
        print()

race:black:labeler=james:validation and race:black:labeler=dan:validation
  black: 272 agreements, 23 disagreements, 11.8:1 ratio, 7.8% disagreement
  not-black: 1795 agreements, 23 disagreements, 78.0:1 ratio, 1.3% disagreement

race:black:labeler=kayvon:validation and race:black:labeler=james:validation
  black: 1 agreements, 285 disagreements, 0.0:1 ratio, 99.7% disagreement
  not-black: 0 agreements, 1804 disagreements, 0.0:1 ratio, 100.0% disagreement

race:black:labeler=kayvon:validation and race:black:labeler=dan:validation
  black: 1 agreements, 280 disagreements, 0.0:1 ratio, 99.6% disagreement
  not-black: 0 agreements, 1809 disagreements, 0.0:1 ratio, 100.0% disagreement

